In [ ]:
#imports
import sys
import subprocess 
import scipy.constants as con
import scipy.optimize as opt
import importlib
#import scipy as sp
import numpy as np
from scipy import special
from scipy import integrate
from scipy.optimize import root_scalar
from scipy.integrate._ivp import common as jacpack
import time
import matplotlib.pyplot as plt

import pynucastro as pyna
reaclibrary = pyna.ReacLibLibrary()

In [ ]:
#Background
#Special functions og deres afledte
def L(z):
    return special.kn(2,z)/z

def M(z):
    return (3/4*special.kn(3,z)+1/4*special.kn(1,z))/z

def dMdz(z):
    return -3/z**2*special.kn(3,z) -special.kn(2,z)/z

def N(z):
    return (1/2*special.kn(4,z)+1/2*special.kn(2,z))/z


timeunit =1.519*10**21  #MeV/hbar in unit of 1/second 

n_life=879.6*timeunit   #Neutron lifetime in units of MeV/hbar
Q=1.293                 #neutron proton mass difference in MeV
M_u=931.494102          #atomic mass unit in MeV
SBC=np.pi**2/60         #Stefan-Boltzmann constant in natural units
G=6.709e-45             #gravatational constant in units of c=hbar=MeV=1
#infapprox=1e3           #large number acting as upper limit on itegrals
n_nu=3.046              #number of neutrino families with correction from 
                        #Nollett and Steigman, BBN and the CMB constrain neutrino coupled light WIMPs, 2015

TMeV2T9=11.60451812 #conversion factor from MeV to 10^9K
cm3s=1.167*10**-11  #conversion factor for cm^3/s
gcm3=232012         #conversion factor for g/cm^3
meter=5.068e12      #conversion factor for m
cm=5.068e10         #conversion factor for cm
barn=389.4          #conversion factor for barn
e_mass=0.51099895   #electron mass in MeV


def rho_e(T): #electron/positron density, chemical potential assumed to be 0 so cosh(phi*n)=1
    z=e_mass/T
    '''
    func=lambda x: np.real(2/(np.pi**2)*e_mass**4*((x**2*(x**2-1)**0.5)/((np.exp(z*x)+1))))

    result, error = integrate.quad(func, 0, infapprox)#[0]
    return result
    '''
    return 2/np.pi**2*e_mass**4*np.sum([(-1)**(n+1)*M(n*z) for n in range(1,10)])

def drho_e(T): #derivative of rho_e with respect to temperature
    z=e_mass/T
    '''
    func=lambda x: np.real(2/(np.pi**2)*e_mass**4*((x**2*(x**2-1)**0.5)*(z*x*np.exp(z*x))/(T*(np.exp(z*x)+1)**2)))

    result, error = integrate.quad(func, 0, infapprox)#[0]
    return result
    '''
    return 2/np.pi**2*e_mass**4*np.sum([(-1)**(n)*n*z/T*dMdz(n*z) for n in range(1,10)])


def P_e(T): #electron/positron pressure
    z=e_mass/T
    '''
    func=lambda x: 2/(3*np.pi**2)*e_mass**4*((x**4*(x**2+1)**-0.5)/((np.exp(z*(x**2+1)**0.5)+1)))

    result, error = integrate.quad(func, 0, infapprox)#[0]
    return result
    '''
    #Fowler and Hoyle 1964
    #return 4/np.pi**2*T**4*np.sum([(-1)**(n+1)/n**4*special.kn(2,n*z) for n in range(1,10)])
    
    #Kawano
    return 2/np.pi**2*e_mass**4*np.sum([(-1)**(n+1)/(n*z)*L(n*z) for n in range(1,10)])  



def rho_gamma(T):   #photon energy density
    return (np.pi**2)/15*T**4 

def drho_gamma(T):  #derivative
    return 4*(np.pi**2)/15*T**3 

def P_gamma(T):     #photon pressure
    return rho_gamma(T)/3



### Initial conditions ###

T_ini=27/TMeV2T9            #initial temperature in MeV

t_ini=1.226*10**21/T_ini**2 #initial time in hbar/MeV
t_cut=1*timeunit            #time at which full reaction network is added

z_ini=e_mass/T_ini

n_gamma_ini= 1.20206*2/np.pi**2*T_ini**3    #initial number density of photons based on theory
n_gamma_ini= rho_gamma(T_ini)/(2.701*T_ini) #initial number density of photons based mean photon energy

eta=6.1e-10     #CMB baryon-to-photon ratio
eta_ini=eta*(1+(rho_e(T_ini)+P_e(T_ini))/(rho_gamma(T_ini)+P_gamma(T_ini))) #entropi bevarelse


rho_nu_ini=n_nu*7/8*(np.pi**2)/15*T_ini**4 #initial neutrino density

h_ini=M_u/T_ini**3*eta_ini*n_gamma_ini  #initial h



rho_b_ini=M_u*eta_ini*n_gamma_ini    #initial baryon density

rho_tot_ini=rho_e(T_ini)+rho_gamma(T_ini)+rho_nu_ini#+rho_b_ini  #initial total energy density

H_ini=np.sqrt(8*np.pi/3*G*rho_tot_ini)


def rho_nu(T,a): #neutrina energy density
    #return 7/8*np.pi**2/15*T**4
    return rho_nu_ini/a**4

def rho_b(a):
    return rho_b_ini/a**3



#def P_b(T,h):
#    return h*T**4*2/3*

#Phie_ini=np.pi**2/2*con.Avogadro*h_ini/z_ini**3*1/sum([(-1)**(n+1)*n*L(n*z_ini) for n in range(1,20)]) 




### Solving the background ###

def rho_tot(T,a): #total density
    return rho_e(T)+rho_gamma(T)+rho_nu(T,a)#+rho_b(a)

def rho_set(T,a): #total density of non-decoupled components
    return rho_e(T)+rho_gamma(T)#+rho_b(a)

def H(T,a):   #Hubble parameter as given by Friedmann eq, ignoring cosmological constant
    return np.sqrt(8*np.pi/3*G*rho_tot(T,a))

#Derivative from Kavano D.18
def dTdt(t,T,a):
    return -3*H(T,a)/((drho_e(T)+drho_gamma(T))/(rho_set(T,a) + P_e(T)+P_gamma(T)))

#Derivative from Kavano D.19
def dhdt(t,T,h,a):
    return -3*h*(H(T,h,a)+dTdt(t,T,h,a)/T)

#Derivative from how H is defined
def dadt(t,T,a):
    return a*H(T,a)


#Derivatives for Jacobian

def dadT(T,a):
    return -((drho_e(T)+drho_gamma(T))/(rho_set(T,a) + P_e(T)+P_gamma(T)))*a/3

def drho_nu(T,a): #neutrina energy density
    return -rho_nu_ini*4/a**5

def drho_b(T,a):
    return -h_ini*T_ini**3*3/a**4

def rho_totdT(T,a): #total density
    return drho_e(T)+drho_gamma(T)+(drho_nu(T,a))#+drho_b(T,a))*dadT(T,a)

def rho_totda(T,a): #total density
    return (drho_e(T)+drho_gamma(T))/dadT(T,a)+drho_nu(T,a)#+drho_b(T,a)



def HdT(T,a):   #Hubble parameter deriv with respect to Temperature
    return 4*np.pi/3*G*rho_totda(T,a)/H(T,a)

def Hda(T,a):   #Hubble parameter deriv with respect to Scale factor
    return 4*np.pi/3*G*rho_totdT(T,a)/H(T,a)

def dadtda(t,T,a):
    return H(T,a)+a*Hda(T,a)


def dadtdT(t,T,a):
    return dadT(T,a)*H(T,a)+a*HdT(T,a)



t_range=[t_ini,5e4*timeunit]            #time range for integration
#t_space=np.linspace(*t_range,1000)      #time range for approximate temperature

#combining derivatives
def dbackground(t,y):   #solve h and T, y[0] = T, y[1] = a
    return [dTdt(t,*y),dadt(t,*y)]



# Solving it
sol = integrate.solve_ivp(dbackground, t_range, [T_ini,1],method='Radau')#,t_eval=t_space)

plt.figure()
# Plot the Temperature
plt.plot(sol.t/timeunit, sol.y[0], label='Photon temperature')

# Test with completely radiation dominated universe
t_space=sol.t
T_approx=np.sqrt(1.226*10**21/t_space)
#T_approx=1/np.sqrt(t_space/timeunit)/10.4*TMeV2T9
plt.plot(t_space/timeunit, T_ini/sol.y[1], label='Neutrino temperature')


#plot the scale factor
#plt.plot(sol.t/timeunit, sol.y[2]/1e3, label='scale factor')

#fix the axis
plt.xlabel('time in seconds')
plt.ylabel('Temperature in MeV')

plt.xscale('log')
plt.yscale('log')

# plt.xlim((0.1, 2e5))
plt.ylim((1e-3,3))

plt.legend()
plt.show()

#plt.figure


n_bparams=2


In [ ]:
#import networks

#section for AoT compilation of network
import AoT_net as bbn_n
nNucs=bbn_n.nnuc()
import full_AoT_net as bbn_full
fnNucs=bbn_full.nnuc()

#section for jit compilation of network
"""
import bbn_test_integrate as bbn_n
nNucs=bbn_n.nnuc
import full_size_net as bbn_full
fnNucs=bbn_full.nnuc

#compile rhs and Jacobians
prev_time=time.time()
bbn_n.rhs(1000, np.ones(nNucs),0.00001, 1e7)
print(f'rhs compiled in {time.time()-prev_time:.2f} seconds')
prev_time=time.time()
bbn_n.jacobian(1000, np.ones(nNucs),0.00001, 1e7)
print(f'jacobian compiled in {time.time()-prev_time:.2f} seconds')
prev_time=time.time()
bbn_full.rhs(1000, np.ones(fnNucs),0.00001, 1e7)
print(f'Full rhs compile in {time.time()-prev_time:.2f} seconds')
prev_time=time.time()
bbn_full.jacobian(1000, np.ones(fnNucs),0.00001, 1e7)
print(f'Full jacobian compiled in {time.time()-prev_time:.2f} seconds')

"""

#importlib.reload(bbn_n)
#importlib.reload(bbn_full)
#import He4_test_integrate as He4_n


In [ ]:
#Setup isotopes based on Alterbbn
Y_labels=['n','p','H2','H3','He3','He4','Li6','Li7','Li8','Be7','Be9','B8','B10','B11','B12','C11','C12','C13','C14','N12','N13','N14','N15','O14','O15','O16']
Alter_Yl=["n","p","H2","H3","He3","He4","Li6","Li7","Be7","Li8","B8","Be9","B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
A=np.array([1,1,2,3,3,4,6,7,8,7,9,8,10,11,12,11,12,13,14,12,13,14,15,14,15,16])
Z=sorted([0,1,1,1,2,2,3,3,4,3,5,4,5,5,6,5,6,7,6,7,6,7,8,7,8,8.])
Alter_A=[1.,1.,2.,3.,3.,4.,6.,7.,7.,8.,8.,9.,10.,11.,11.,12.,12.,12.,13.,13.,14.,14.,14.,15.,15.,16.]
Alter_Z=[0.,1.,1.,1.,2.,2.,3.,3.,4.,3.,5.,4.,5.,5.,6.,5.,6.,7.,6.,7.,6.,7.,8.,7.,8.,8.]
print()

#new nuclei Be11 he6 c10

def Altersort(L):
    if type(L)==list:
        return [x for (a,x) in sorted(zip(zip(A,Z),L), key=lambda pair: pair[0])]
    else:
        return np.array([x for (a,x) in sorted(zip(zip(A,Z),L), key=lambda pair: pair[0])])
#print([label for _, label in sorted(zip(A, Y_labels))])

def PNAsort(L):
    if type(L)==list:
        return [x for (a,x) in sorted(zip(zip(Alter_Z,Alter_A),L), key=lambda pair: pair[0])]
    else:
        return np.array([x for (a,x) in sorted(zip(zip(Alter_Z,Alter_A),L), key=lambda pair: pair[0])])    


print(Alter_Yl)
print(Altersort(Y_labels))

print(Y_labels)
print(PNAsort(Alter_Yl))

In [ ]:
#Initial conditions for Y

Alter_Yl=["n","p","H2","H3","He3","He4","Li6","Li7","Be7","Li8","B8","Be9","B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
Alter_A=[1.,1.,2.,3.,3.,4.,6.,7.,7.,8.,8.,9.,10.,11.,11.,12.,12.,12.,13.,13.,14.,14.,14.,15.,15.,16.]
Alterspin=[0.5,0.5,1.,0.5,0.5,0.,1.,1.5,1.5,2.,2.,1.5,3.,1.5,1.5,1.,0.,1.,0.5,0.5,0.,1.,0.,0.5,0.5,0.]
Alter_mass_excess=[8.071388,7.289028,13.135825,14.949915,14.931325,2.424931,14.0864,14.9078,15.7696,20.9464,22.9212,11.34758,12.05086,8.6680,10.6506,13.3690,0.,17.3382,3.125036,5.3455,3.019916,2.863440,8.006521,0.101439,2.8554,-4.737036]
Alter_mass=[Alter_A[i]*M_u+Alter_mass_excess[i] for i in range(fnNucs)]

spin=PNAsort(Alterspin)
Y_labels=['n','p','H2','H3','He3','He4','Li6','Li7','Li8','Be7','Be9','B8','B10','B11','B12','C11','C12','C13','C14','N12','N13','N14','N15','O14','O15','O16']
A=np.array(PNAsort(Alter_A))


m_Nucs = np.array(PNAsort(Alter_mass))



def rho_bY_cgs(y):
    return sum(m_Nucs[:len(y[n_bparams:])]*y[n_bparams:])*eta_ini*n_gamma_ini/y[1]**3*gcm3

#equilibrium ratios for simple rates #Pebbles
m_n,m_p,m_d,m_t,m_He3,m_He4=m_Nucs[:6]

def G_np(T,a):
    N=eta_ini*n_gamma_ini/a**3 #number density
    return 4/3*(m_n*m_p*T/(2*np.pi*m_d))**1.5/N*np.exp((m_d-m_n-m_p)/T)

def G_nHe3(T):
    return 9/4*(m_d**2/(m_He3*m_n))**1.5*np.exp((m_n+m_He3-2*m_d)/T)

def G_tp(T):
    return 9/4*(m_d**2/(m_t*m_p))**1.5*np.exp((m_p+m_t-2*m_d)/T)  

def G_tp_nHe3(T):
    return ((m_He3*m_n)/(m_t*m_p))**1.5*np.exp((m_p+m_t-(m_n+m_He3))/T)

def G_nHe4(T):
    return 3*((m_d+m_t)/(m_He4*m_n))**1.5*np.exp((m_He4+m_n-m_d-m_t)/T)  

#Initial conditions

Y_ini=np.zeros(nNucs)+1e-80
#Y_ini=np.array([1e-20 for i in range(nNucs)])

Xn_ini = 1/(np.exp(Q/T_ini)+1)

#Initial condition based on thermal equilibrium amongst in the most important reactions

Y_ini[0] = Xn_ini        #Set initial neutron mass fraction
Y_ini[1] = 1 - Xn_ini    #Set initial proton mass fraction
Y_ini[2] = Y_ini[0]*Y_ini[1]/G_np(T_ini,1)          #Set initial deuterium mass fraction
Y_ini[3] = Y_ini[2]**2/Y_ini[1]/G_tp(T_ini)
Y_ini[4] = Y_ini[2]**2/Y_ini[0]/G_nHe3(T_ini)
Y_ini[5] = Y_ini[2]*Y_ini[3]/Y_ini[0]/G_nHe4(T_ini)




dYdt_ini=lambda Y : bbn_n.rhs(t_ini/timeunit, Y ,rho_bY_cgs([T_ini,1]+list(Y)), T_ini*TMeV2T9*1e9)


In [ ]:
#Initial conditions from thermal equilibrium

def get_Y_ini(Xn_ini):
    Y_ini2 = np.zeros(nNucs)+1e-50
    #Y_ini=np.array([1e-20 for i in range(nNucs)])


    #Xn_ini = 1/(np.exp(Q/T_ini)+1)
    

    B=[(m_Nucs[1]*Z[i]+m_Nucs[0]*(A[i]-Z[i]))-m_Nucs[i] for i in range(nNucs)]
    g = 1+2*np.array(spin)
        
    for iter1 in range(30):
        Xp_ini = np.exp(Q/T_ini)*Xn_ini
        Y_ini2[0] = Xn_ini        #Set initial neutron mass fraction
        Y_ini2[1] = Xp_ini
        #Set initial proton mass fraction
        X_sum = Y_ini2[0] + Y_ini2[1]
        for i in range(2,nNucs):
            tmp = special.zeta(3)**(A[i] - 1)*np.pi**((1 - A[i])/2)*2**((3*A[i] - 5)/2)*A[i]**(5/2)
            #print(f'A({key}) = {A} : {g[key]*tmp}')
            Y_ini2[i] = g[i]*tmp*(T_ini/m_n)**(3*(A[i] - 1)/2)*eta_ini**(A[i] - 1)*Xp_ini**Z[i]*Xn_ini**(A[i] - Z[i])*np.exp(B[i]/T_ini)/A[i]
            X_sum += Y_ini2[i]*A[i]
        #print(X_sum)
        Xn_ini += (1 - X_sum)/nNucs
        
    return Y_ini2
Xn_ini = 1/(np.exp(Q/T_ini)+1)
Y_ini2 = get_Y_ini(Xn_ini)

    #B12 = 92.2 

#Initial condition based on thermal equilibrium amongst in the most important reactions
B2 = 2.22452
B3 = 7.72
B4 = 28.295673


print(f'From initial guess:')
print('\t'.join([f'{yy:.2e}' for yy in Y_ini]))
print('\t'.join([f'{yy:.2e}' for yy in dYdt_ini(Y_ini)]))
print(f'From equilibrium conditions:')
print('\t'.join([f'{yy:.2e}' for yy in Y_ini2]))
print('\t'.join([f'{yy:.2e}' for yy in dYdt_ini(Y_ini2)]))


In [ ]:
#initial conditions from Jacobian
AdYdt_ini=lambda Y : Altersort(bbn_n.rhs(t_ini/timeunit, PNAsort(Y) ,rho_bY_cgs([T_ini,1]+list(PNAsort(Y))), T_ini*TMeV2T9*1e9))
AdYdt_jac=lambda Y : bbn_n.jacobian(t_ini/timeunit, PNAsort(Y) ,rho_bY_cgs([T_ini,1]+list(PNAsort(Y))), T_ini*TMeV2T9*1e9)[:, Altersort(range(nNucs))][Altersort(range(nNucs))]
import scipy.linalg as la
aY_ini=Altersort(Y_ini2)
def solve_using_svd(U, s, Vh, b):
    bb = U.T @ b
    y = bb/s
    x = Vh.T @ y
    return x
Yj = np.array([YY for YY in aY_ini])
cut_start = 0
for cut in range(cut_start, len(Yj) - 2, 1):
    for j in range(10):
        print(cut, j)
        print(Yj)
        fyj = -AdYdt_ini(Yj)
        jac = AdYdt_jac(Yj)
        if np.any(np.isnan(Yj)) or np.any(np.isinf(Yj)):
            print('Yj:', Yj)
            raise ValueError
        if np.any(np.isnan(fyj)) or np.any(np.isinf(fyj)):
            print('Yj:', Yj)
            print('fyj:', fyj)
            raise ValueError
        # Implement cut:
        fyj = fyj[cut:]
        jac = jac[cut:, cut:]
        
        # Solution using SVD
        U, s, Vh = la.svd(jac)    
        x = solve_using_svd(U, s, Vh, fyj)
        for k in range(1):
            #A · δx = A · (x + δx) − b
            residuals = jac @ x - fyj
            dx = solve_using_svd(U, s, Vh, residuals)
            x -= dx
        Yj[cut:] += x


def print_stuff(YY, name=''):
    print(f'From {name}:')
    print('\t'.join([f'{yy:.2e}' for yy in YY]))
    print('\t'.join([f'{yy:.2e}' for yy in AdYdt_ini(YY)]))
print_stuff(Yj, 'manual tuning')
print_stuff(aY_ini, 'equilibrium conditions')


In [ ]:
#Combining background and network


initial_param=[T_ini,1]+list(PNAsort(Yj))

n_params=nNucs+n_bparams


def ndall(t,y):   #solve h and T, y[0] = T and y[1] = a   
    #print(ndbackground(t,y[:n_bparams])+list( bbn_n.rhs(t/timeunit, y[n_bparams:],eta_ini*n_gamma_ini/y[2]**3*m_p*gcm3, y[0]*TMeV2T9*1e9)/timeunit))
    result=dbackground(t,y[:n_bparams])+list( bbn_n.rhs(t/timeunit, y[n_bparams:],rho_bY_cgs(y), y[0]*TMeV2T9*1e9)/timeunit)
    #testdata.append([t]+result)                 
    return result


In [ ]:
#solving with analytic jacobian




def jacY_anal(t,y):
    return bbn_n.jacobian(t/timeunit, y[n_bparams:],rho_bY_cgs(y), y[0]*TMeV2T9*1e9)/timeunit

def jacfun(t,x):
    return np.array([ndall(t,row)  for row in x.T]).T

Factor=None
def jacobian(t,y):
    '''
    global Factor
    jac_rhs_value=np.array(ndall(t,y))
    jactest=jacpack.num_jac(jacfun,t,y,jac_rhs_value,1e-30,Factor)
    Factor=jactest[1]
    jactest[0][:,:] = np.zeros((n_params,n_params))
    jactest[0][2:,:] = np.zeros((nNucs,n_params))
    jactest[0][:,2:] = np.zeros((n_params,nNucs))
    jactest[0][2:,2:]=jacY_anal(t,y)
    
    
    return jactest[0]
    '''
    #print(np.shape(y))
    jac=np.append(np.zeros((nNucs,n_bparams)),jacY_anal(t,y),axis=1)
    return np.append(np.zeros((n_bparams,n_params)),jac,axis=0)
    
    
    

jacsolY = integrate.solve_ivp(ndall, [0,t_cut-t_ini], initial_param,method='Radau',atol=1e-80,rtol=1e-6,jac=jacobian)#,t_eval=t_space)

print(jacsolY.message)
print(jacsolY.nfev)
print(jacsolY.njev)
print(jacsolY.nlu)

abun=[abun[-1] for abun in jacsolY.y[n_bparams:]]


print([abun[0],abun[1],2*abun[2],3*abun[3],3*abun[4],4*abun[5]])
print('sum: ', sum(np.array(abun)*A[:nNucs]))

In [ ]:
#cut conditions from thermal equilibrium
T_cut=jacsolY.y[0][-1]

dYdt_cut=lambda Y : bbn_full.rhs(t_cut/timeunit, Y ,rho_bY_cgs([param[-1] for param in jacsolY.y]), T_cut*TMeV2T9*1e9)

Y_cut=np.zeros(fnNucs)+1e-80
Y_cut[:nNucs] = abun

def get_Y_cut(abun_cut):
    Y_ini3 = np.zeros(fnNucs)+1e-80
    #Y_ini=np.array([1e-20 for i in range(nNucs)])


    #Xn_ini = 1/(np.exp(Q/T_ini)+1)
    

    B=[(m_Nucs[1]*Z[i]+m_Nucs[0]*(A[i]-Z[i]))-m_Nucs[i] for i in range(fnNucs)]
    g = 1+2*np.array(spin)
        
    for iter1 in range(30):
        
        Xn_cut = abun_cut[0]
        Xp_cut = abun_cut[1]
        Y_ini3[:nNucs] = abun_cut       #Set initial neutron mass fraction
        
        #Set initial proton mass fraction
        #X_sum = Y_ini3[0] + Y_ini3[1]
        for i in range(nNucs,fnNucs):
            tmp = special.zeta(3)**(A[i] - 1)*np.pi**((1 - A[i])/2)*2**((3*A[i] - 5)/2)*A[i]**(5/2)
            #print(f'A({key}) = {A} : {g[key]*tmp}')
            Y_ini3[i] = g[i]*tmp*(T_ini/m_n)**(3*(A[i] - 1)/2)*eta_ini**(A[i] - 1)*Xp_cut**Z[i]*Xn_cut**(A[i] - Z[i])*np.exp(B[i]/T_cut)/A[i]
            #X_sum += Y_ini3[i]*A[i]
        #print(X_sum)
        #Xn_ini += (1 - X_sum)/nNucs
        
    return Y_ini3

Y_cut2 = get_Y_cut(abun)

    #B12 = 92.2 

#Initial condition based on thermal equilibrium amongst in the most important reactions
B2 = 2.22452
B3 = 7.72
B4 = 28.295673


print(f'From initial guess:')
print('\t'.join([f'{yy:.2e}' for yy in Y_cut]))
print('\t'.join([f'{yy:.2e}' for yy in dYdt_cut(Y_cut)]))
print(f'From equilibrium conditions:')
print('\t'.join([f'{yy:.2e}' for yy in Y_cut2]))
print('\t'.join([f'{yy:.2e}' for yy in dYdt_cut(Y_cut2)]))


In [ ]:
#cut conditions from Jacobian
AdYdt_cut=lambda Y : Altersort(bbn_full.rhs(t_cut/timeunit, PNAsort(Y) ,rho_bY_cgs([param[-1] for param in jacsolY.y]), T_cut*TMeV2T9*1e9))
AdYdt_jac_cut=lambda Y : bbn_full.jacobian(t_cut/timeunit, PNAsort(Y) ,rho_bY_cgs([param[-1] for param in jacsolY.y]), T_cut*TMeV2T9*1e9)[:, Altersort(range(fnNucs))][Altersort(range(fnNucs))]
import scipy.linalg as la
aY_cut=Altersort(Y_cut2)
def solve_using_svd(U, s, Vh, b):
    bb = U.T @ b
    y = bb/s
    x = Vh.T @ y
    return x
Yj = np.array([YY for YY in aY_cut])
cut_start = nNucs
for cut in range(cut_start, len(Yj) - 2, 1):
    for j in range(10):
        print(cut, j)
        print(Yj)
        fyj = -AdYdt_cut(Yj)
        jac = AdYdt_jac_cut(Yj)
        if np.any(np.isnan(Yj)) or np.any(np.isinf(Yj)):
            print('Yj:', Yj)
            raise ValueError
        if np.any(np.isnan(fyj)) or np.any(np.isinf(fyj)):
            print('Yj:', Yj)
            print('fyj:', fyj)
            raise ValueError
        # Implement cut:
        fyj = fyj[cut:]
        jac = jac[cut:, cut:]
        
        # Solution using SVD
        U, s, Vh = la.svd(jac)    
        x = solve_using_svd(U, s, Vh, fyj)
        for k in range(1):
            #A · δx = A · (x + δx) − b
            residuals = jac @ x - fyj
            dx = solve_using_svd(U, s, Vh, residuals)
            x -= dx
        Yj[cut:] += x


def print_stuff(YY, name=''):
    print(f'From {name}:')
    print('\t'.join([f'{yy:.2e}' for yy in YY]))
    print('\t'.join([f'{yy:.2e}' for yy in AdYdt_cut(YY)]))
print_stuff(Yj, 'manual tuning')
print_stuff(aY_cut, 'equilibrium conditions')


In [ ]:
#Combining background and full network

cut_param=[param[-1] for param in jacsolY.y[:n_bparams]]+list(PNAsort(Yj))

t_range_cut=[0,1e5*timeunit-t_cut+t_ini]

full_n_params=fnNucs+n_bparams


def ndbackground(t,y):   #solve h and T, y[0] = T, y[1] = h and y[2] = a
    return [dTdt(t,*y),dadt(t,*y)]


#testdata=[]

def ndfull(t,y):   #solve h and T, y[0] = T and y[1] = a   
    #print(ndbackground(t,y[:n_bparams])+list( bbn_n.rhs(t/timeunit, y[n_bparams:],eta_ini*n_gamma_ini/y[2]**3*m_p*gcm3, y[0]*TMeV2T9*1e9)/timeunit))
    result=ndbackground(t,y[:n_bparams])+list( bbn_full.rhs(t/timeunit, y[n_bparams:],rho_bY_cgs(y), y[0]*TMeV2T9*1e9)/timeunit)
    #testdata.append([t]+result)                 
    return result


In [ ]:
#solving with full network




def full_jac(t,y):
    return bbn_full.jacobian(t/timeunit, y[n_bparams:],rho_bY_cgs(y), y[0]*TMeV2T9*1e9)/timeunit


Factor=None
def full_jacobian(t,y):
    #print(np.shape(y))
    jac=np.append(np.zeros((fnNucs,n_bparams)),full_jac(t,y),axis=1)
    return np.append(np.zeros((n_bparams,full_n_params)),jac,axis=0)
    
    
    

fullsolY = integrate.solve_ivp(ndfull, t_range_cut, cut_param,method='Radau',atol=1e-80,rtol=1e-6,jac=full_jacobian)#,t_eval=t_space)

print(fullsolY.message)
print(fullsolY.nfev)
print(fullsolY.njev)
print(fullsolY.nlu)

final_abun=[fabun[-1] for fabun in fullsolY.y[n_bparams:]]


print([final_abun[0],final_abun[1],2*final_abun[2],3*final_abun[3],3*final_abun[4],4*final_abun[5]])
print('sum: ', sum(np.array(final_abun)*A))

In [ ]:
#Combine the high temperature and and full network solutions
solY=np.concatenate((np.concatenate((jacsolY.y,np.multiply(np.ones((len(jacsolY.y[0]),fnNucs-nNucs)),cut_param[n_params:]).T)),fullsolY.y),axis=1)

soltime=np.concatenate((jacsolY.t+t_ini,fullsolY.t+t_cut))/timeunit

In [ ]:
# Plot the results    

plt.figure('abundance',figsize=(6.4, 8))
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors*=3
line=['-']*10+['--']*10+[':']*10

for i in range(fnNucs):
      plt.plot(range(len(soltime)), A[i]*solY[n_bparams+i],line[i], color=colors[i], label=Y_labels[i])

plt.xlabel('step')
plt.ylabel('Mass fraction')
plt.ylim(1e-40,1)
#plt.xlim(0,20)
#plt.xscale('log')
plt.yscale('log')
plt.legend(loc='right')
plt.show()

plt.figure('step size')
plt.plot(range(len(soltime)),[(soltime[i]-soltime[i-1]) for i in range(len(soltime))] )
plt.yscale('log')
plt.xlabel('step')
plt.ylabel('stepsize in seconds')


plt.figure('abundance',figsize=(6.4, 8))
for i in range(fnNucs):
    plt.plot(soltime, A[i]*solY[n_bparams+i],line[i], color=colors[i], label=Y_labels[i])

plt.xlabel('Time in seconds')
plt.ylabel('Mass fraction')
plt.ylim(1e-20,3)
plt.xlim(t_ini/timeunit,1e5)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
#find alterbbn abundance
result = subprocess.run(["/home/hansbdein/Speciale/alterbbn_v2.2/stand_cosmo.x", '6'], capture_output=True)

In [ ]:
#compare to Alterbbn abundance
abun=final_abun
print('Observational values')
print('\t Yp  '+'\t\t H2/H '+'\t\t H3/H '+'\t\t Li7/H ')
print('value:\t 2.450e-01 \t 2.527e-05 \t 1.1e-05 \t 1.6e-10')
print(' +/-  \t 0.030e-01  \t 0.030e-05 \t 0.2e-05 \t 0.3e-10')

print(' ')
print('Results from this code')
print('\t Yp  '+'\t\t H2/H '+'\t\t H3/H '+'\t\t Li7/H '+'\t\t Li6/H '+'\t\t Be7/H ')

print('value:\t '"{:.3e}".format(4*abun[5])+'\t '+"{:.3e}".format(abun[2]/abun[1])+'\t '+"{:.3e}".format((abun[3]+abun[4])/abun[1])+'\t '+"{:.3e}".format((abun[7]+abun[9])/abun[1])+'\t '+"{:.3e}".format((abun[6])/abun[1])+'\t '+"{:.3e}".format((abun[9])/abun[1]))


print(' ')
print('Results from Alterbbn, with theoretical uncertainties:')
for info in result.stdout.decode("utf-8").split('\n')[8:11]:
    print(info)
    #print('lmao')

alterbbn_abun=np.array([float(result.stdout.decode("utf-8").split('\n')[9].split('\t')[i]) for i in range(1,7)])

reorderabun=[4*abun[5],abun[2]/abun[1],(abun[3]+abun[4])/abun[1],(abun[7]+abun[9])/abun[1],(abun[6])/abun[1],(abun[9])/abun[1]]

ratio_Str='ratio:'
for i in range(6):
    ratio_Str+='\t '+ '{:g}'.format(float(result.stdout.decode("utf-8").split('\n')[9].split('\t')[i+1])/reorderabun[i])
print(' ')
print('ratio of this code to ALterbbn results:')
print(ratio_Str)

In [ ]:
abun[7]/2.2687415497617955e-11

In [ ]:
#Load precise Alterbbn data
data=np.loadtxt('./Alterdata/output.txt',skiprows=2)

'''
#run Alterbbn with chosen integration routine
failsafe=12
subprocess.run(["/home/hansbdein/Speciale/alterbbn_v2.2/no_err_calc.x", str(failsafe)], capture_output=True)
data=np.loadtxt('./output.txt',skiprows=2)
'''


#remove non-time steps
prevtime=0
mask=[]
for timestep in data:
    if timestep[0]>prevtime:
        prevtime=timestep[0]
        mask.append(True)
    else:
        mask.append(False)


data=data[mask,...]

print(data[0,0]/1e3)
#151983983107661.34
#1.519839831076613438e+14
#2.264733810941325e+20

#change Alterbbn data for time and temperature in hbar/GeV to hbar/MeV
data[:,0]*=1e-3
data[:,1]*=1e3
data[:,2]*=1e3
#change order of isotopes to be sorted by Y rather than A
data[:,3:]=PNAsort(data[:,3:].T).T
print(t_ini)

In [ ]:
# Plot the results
  
plt.figure('abundance',figsize=(6.4, 8))
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors*=3

for i in range(fnNucs):
    plt.plot(soltime, A[i]*solY[n_bparams+i],color=colors[i], label=Y_labels[i])
    plt.plot(data[:,0]/timeunit, A[i]*data[:,i+3],':', color=colors[i], label='Alterbbn_'+Y_labels[i])


plt.xlabel('time')
plt.ylabel('Mass fraction')
plt.ylim(1e-30,1)
plt.xlim(t_ini/timeunit ,1e5)
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='center left')
plt.show()

In [ ]:
t_space=[point for point in data[:,0] if point<t_cut]-data[0,0]

alterjacsolY = integrate.solve_ivp(ndall, [0,t_cut-t_ini], initial_param,method='Radau',atol=1e-80,rtol=1e-6,jac=jacobian,t_eval=t_space)

ft_space=[point for point in data[:,0] if point>t_cut]
ft_space-=ft_space[0]
alterfullsolY = integrate.solve_ivp(ndfull, [0,ft_space[-1]], cut_param,method='Radau',atol=1e-80,rtol=1e-6,jac=full_jacobian,t_eval=ft_space)


In [ ]:
#Combine the high temperature and and full network solutions
altersolY=np.concatenate((np.concatenate((alterjacsolY.y,np.multiply(np.ones((len(alterjacsolY.y[0]),fnNucs-nNucs)),cut_param[n_params:]).T)),alterfullsolY.y),axis=1)

altersoltime=np.concatenate((alterjacsolY.t+t_ini,alterfullsolY.t+t_cut))/timeunit

In [ ]:
# Plot the results
plt.figure('abundance')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors*=3
line=['-']*10+['--']*10+[':']*10

for i in range(fnNucs):
    plt.plot(data[:,0]/timeunit, data[:,i+3]/altersolY[n_bparams+i],line[i],color=colors[i], label=Y_labels[i])

plt.xlabel('time in seconds')
plt.ylabel('abundance ratio')

plt.xscale('log')


plt.xlim((data[0,0]/timeunit, 2e5))
plt.ylim(0,2)

plt.legend(loc='upper right')
plt.show()

In [ ]:
#plt.plot(data[:,0]/timeunit, (SolAltertime.y[n_bparams+i]-data[:,i+3])/data[:,i+3], label=' (Y - Y_alterbbn)/Y_alterbbn')
#plt.plot(data[:,0]/timeunit, (data[:,0]/timeunit/1e2)**-10*A[5]*data[:,5+3],':', label='Alterbbn_Helium4')
#plt.axhline(0,color='r',linestyle='--',label='0')

for i in range(fnNucs):
    plt.plot(data[:,0]/timeunit, (altersolY[n_bparams+i]-data[:,i+3])/data[:,i+3],line[i],color=colors[i], label=Y_labels[i])

plt.xlabel('time in seconds')
plt.ylabel('difference in mass fraction')

plt.xscale('log')
plt.axhline(0,color='k',linestyle=':',label='0')

plt.xlim((data[0,0]/timeunit, 2e5))
plt.ylim(-1,1)

plt.legend(loc='center left')
plt.show()

In [ ]:
#compare to Alterbbn abundance
abun=[aabun[-1] for aabun in altersolY[n_bparams:]]

print('Observational values')
print('\t Yp  '+'\t\t H2/H '+'\t\t H3/H '+'\t\t Li7/H ')
print('value:\t 2.450e-01 \t 2.569e-05 \t 1.1e-05 \t 1.6e-10')
print(' +/-  \t 0.030e-01  \t 0.027e-05 \t 0.2e-05 \t 0.3e-10')

print(' ')
print('Results from this code')
print('\t Yp  '+'\t\t H2/H '+'\t\t H3/H '+'\t\t Li7/H '+'\t\t Li6/H '+'\t\t Be7/H ')

print('value:\t '"{:.3e}".format(4*abun[5])+'\t '+"{:.3e}".format(abun[2]/abun[1])+'\t '+"{:.3e}".format((abun[3]+abun[4])/abun[1])+'\t '+"{:.3e}".format((abun[7]+abun[9])/abun[1])+'\t '+"{:.3e}".format((abun[6])/abun[1])+'\t '+"{:.3e}".format((abun[9])/abun[1]))


print(' ')
print('Results from Alterbbn, with theoretical uncertainties:')
for info in result.stdout.decode("utf-8").split('\n')[8:11]:
    print(info)
    #print('lmao')

alterbbn_abun=np.array([float(result.stdout.decode("utf-8").split('\n')[9].split('\t')[i]) for i in range(1,7)])

reorderabun=[4*abun[5],abun[2]/abun[1],(abun[3]+abun[4])/abun[1],(abun[7]+abun[9])/abun[1],(abun[6])/abun[1],(abun[9])/abun[1]]
abun=data[-1,3:]
Alterabun=[4*abun[5],abun[2]/abun[1],(abun[3]+abun[4])/abun[1],(abun[7]+abun[9])/abun[1],(abun[6])/abun[1],(abun[9])/abun[1]]


ratio_Str='ratio:'
for i in range(6):
    ratio_Str+='\t '+ '{:g}'.format(float(result.stdout.decode("utf-8").split('\n')[9].split('\t')[i+1])/reorderabun[i])
print(' ')
print('Ratio of this code to ALterbbn results:')
print(ratio_Str)

print(' ')
print('High precision Alterbbn results')
print('\t Yp  '+'\t\t H2/H '+'\t\t H3/H '+'\t\t Li7/H '+'\t\t Li6/H '+'\t\t Be7/H ')

print('value:\t '"{:.3e}".format(4*abun[5])+'\t '+"{:.3e}".format(abun[2]/abun[1])+'\t '+"{:.3e}".format((abun[3]+abun[4])/abun[1])+'\t '+"{:.3e}".format((abun[7]+abun[9])/abun[1])+'\t '+"{:.3e}".format((abun[6])/abun[1])+'\t '+"{:.3e}".format((abun[9])/abun[1]))


ratio_Str='ratio:'
for i in range(6):
    ratio_Str+='\t '+ '{:g}'.format(Alterabun[i]/reorderabun[i])
print(' ')
print('Ratio of this code to high precision Alterbbn results:')
print(ratio_Str)

In [ ]:
final_abun[10]/abun[10]

In [ ]:
#finding rates that contribute to Berylium7 production
all_nuclei = Y_labels[:fnNucs]#["p","n", "h2", "h3", "he3", "he4","Li6","Li7","Be7","Li8"]#,"B8","Be9","B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
bbn_library = reaclibrary.linking_nuclei(all_nuclei)
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)

Li7rates = bbn_library.filter(pyna.RateFilter(products=["Li7"], exact=False)).get_rates()
Li7reverserates = bbn_library.filter(pyna.RateFilter(reactants=["Li7"], exact=False)).get_rates()

filter = pyna.RateFilter(products=["Be7"], exact=False)
Be7rates = bbn_library.filter(filter).get_rates()
reversefilter = pyna.RateFilter(reactants=["Be7"], exact=False)
Be7reverserates = bbn_library.filter(reversefilter).get_rates()
print(Be7rates)
print(Be7reverserates)

In [ ]:
#Function for evaluating pyna rates at a given point
def ratevalue(rate,i):
    Tpeak=fullsolY.y[0,i]*TMeV2T9*1e9
    rho=rho_bY_cgs(fullsolY.y[:,i])
    value=rate.eval(Tpeak)/rho
    if rate.__str__()=='Be7 + e⁻ ⟶ Li7 + 𝜈':
        value*=1*rho
    for reactant in rate.reactants:
        index=2+Y_labels.index(reactant.caps_name)
        value*=fullsolY.y[index,i]*rho
        

    return  value

In [ ]:
fullsolY.t[1400]

In [ ]:
rho_bY_cgs(fullsolY.y[:,1400])

In [ ]:
rho_e(0.06594380559629866)/rho_b(0.06594380559629866)

In [ ]:
0.06594380559629866

In [ ]:
def ye(Y):
    return np.sum(Z * Y)/np.sum(A * Y)

In [ ]:

    print('lmao')

In [ ]:
#Li6 + n ⟶ Li7

#alter
#5100

#this
#exp( 9.04782)=8500
2+Y_labels.index('Be7')


In [ ]:
#-rho*ye(Y)*Y[jbe7]*rate_eval.be7__li7__weak__electron_capture
#-rho*Y[jn]*Y[jbe7]*rate_eval.n_be7__p_li7


In [ ]:
'''
def be7__li7__weak__electron_capture(rate_eval, tf):
    # be7 --> li7
    rate = 0.0

    #   ecw
    rate += np.exp(  -23.8328 + 3.02033*tf.T913
                  + -0.0742132*tf.T9 + -0.00792386*tf.T953 + -0.650113*tf.lnT9)

    rate_eval.be7__li7__weak__electron_capture = rate

def n_be7__p_li7(rate_eval, tf):
    # be7 + n --> p + li7
    rate = 0.0

    # db18 
    rate += np.exp(  21.7899 + 0.000728098*tf.T9i + -0.30254*tf.T913i + -0.3602*tf.T913
                  + 0.17472*tf.T9 + -0.0223*tf.T953 + -0.4581*tf.lnT9)

    rate_eval.n_be7__p_li7 = rate
    '''

In [ ]:
plt.figure(figsize=(6.4, 8))
for ii,rate in enumerate(Be7reverserates):
    #print(rate)
    plt.plot((fullsolY.t+t_cut)/timeunit,[ratevalue(rate,i) for i in range(len(fullsolY.t))],line[ii],color=colors[ii],label=rate)

#plt.plot((fullsolY.t+t_cut)/timeunit,[5100*fullsolY.y[8,i]*fullsolY.y[2,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))],'--k',label='Alter Li6->Li7')

plt.legend(loc='center left')
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('dY/dt')
#plt.yscale('log')
plt.ylim(1e-70,1.5e-11)

In [ ]:
plt.figure(figsize=(6.4, 8))
for ii,rate in enumerate(Be7reverserates[:1]):
    #print(rate)fullsolY.y[0,i]*TMeV2T9*1e9
    plt.plot((fullsolY.t+t_cut)/timeunit,[ratevalue(rate,i) for i in range(len(fullsolY.t))],line[ii],color=colors[ii],label=rate)

#plt.plot((fullsolY.t+t_cut)/timeunit,[5100*fullsolY.y[8,i]*fullsolY.y[2,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))],'--k',label='Alter Li6->Li7')

plt.legend(loc='center left')
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('dY/dt')
#plt.yscale('log')
#plt.ylim(1e-70,1.5e-11)

In [ ]:
for ii,rate in enumerate(Be7reverserates[:1]):
    #print(rate)fullsolY.y[0,i]*TMeV2T9*1e9
    plt.plot((fullsolY.t+t_cut)/timeunit,[rate.eval(fullsolY.y[0,i]*TMeV2T9*1e9) for i in range(len(fullsolY.t))],line[ii],color=colors[ii],label=rate)

    plt.legend(loc='center left')
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('dY/dt')

In [ ]:
plt.figure(figsize=(6.4, 8))
for ii,rate in enumerate(Be7rates):
    #print(rate)
    plt.plot((fullsolY.t+t_cut)/timeunit,[ratevalue(rate,i) for i in range(len(fullsolY.t))],line[ii],color=colors[ii],label=rate)


plt.legend()
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('dY/dt')
#plt.yscale('log')
plt.ylim(1e-70,1.5e-11)

In [ ]:
#H3 + a -> g + Li7 */     // Serpico et al 2004
def AlterLi7(T9):
    if (T9<2.5):
        rate=((0.094614248-4.9273133*T9+99.358965*T9*T9-989.81236*T9*T9*T9+4368.45*pow(T9,4.)+931.93597*pow(T9,5.)
                -391.07855*pow(T9,6.)+159.23101*pow(T9,7.)-34.407594*pow(T9,8.)+3.3919004*pow(T9,9.)
                +0.017556217*pow(T9,10.)-0.036253427*pow(T9,11.)+0.0031118827*pow(T9,12.)
                -0.00008714468*pow(T9,13.))*pow(T9,-1./2.))/(np.exp(8.4e-7*T9)*pow(1.+1.78616593*T9,3.))
    
    else: rate=807.406
    return rate

def AlterLi7d(T9):
    if (T9<2.5):
        rate=((-8.9654123e7-2.5851582e8*T9-2.6831252e7*T9*T9+3.8691673e8*pow(T9,1./3.)+4.9721269e8*pow(T9,2./3.)
                +2.6444808e7*pow(T9,4./3.)-1.2946419e6*pow(T9,5./3.)-1.0941088e8*pow(T9,7./3.)
                +9.9899564e7*pow(T9,8./3.))*pow(T9,-2./3.))/np.exp(7.73389632*pow(T9,-1./3.))
        rate+=(np.exp(-1.137519e0*T9*T9-8.6256687*pow(T9,-1./3.))*(3.0014189e7-1.8366119e8*T9+1.7688138e9*T9*T9
                                                                -8.4772261e9*T9*T9*T9+2.0237351e10*pow(T9,4.)
                                                                -1.9650068e10*pow(T9,5.)+7.9452762e8*pow(T9,6.)
                                                                +1.3132468e10*pow(T9,7.)-8.209351e9*pow(T9,8.)
                                                                -9.1099236e8*pow(T9,9.)+2.7814079e9*pow(T9,10.)
                                                                -1.0785293e9*pow(T9,11.)
                                                                +1.3993392e8*pow(T9,12.))*pow(T9,-2./3.))
    else:
        rate=1.53403e6
        rate+=84516.7
    return rate

def AlterBe7(T9):
    if(T9<2.5):
        rate=(np.exp(15.609867-(12.82707707/pow(T9,1./3.))-0.6666667*np.log(T9))*
        ((1-0.020478*pow(T9,2./3.)+0.211995*pow(T9,4./3.))/(1+0.255059*pow(T9,2./3.)+0.338573*pow(T9,4./3.))))
    else: rate=0

    return rate



def AlterBe7d(T9):
    if(T9<2.5):
        rate=(6.8423032e9+1.7674863e10*T9+2.6622006e9*T9*T9-3.3561608e8*T9*T9*T9-5.9309139e6*pow(T9,4.)
        -1.4987996e10*np.sqrt(T9)-1.0576906e10*pow(T9,3./2.)+2.7447598e8*pow(T9,5./2.)
        +7.6425157e7*pow(T9,7./2.)-2.282944e7*pow(T9,-3./2.) / np.exp(0.050351813/T9))

    else: rate=1.28039e9
    return rate





In [ ]:
fullsolY.y[3,-1]

In [ ]:
mainLi7=np.array([Li7rates[3].eval(fullsolY.y[0,i]*TMeV2T9*1e9) * fullsolY.y[5,i]*fullsolY.y[7,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])
Alter_mainLi7=np.array([AlterLi7(fullsolY.y[0,i]*TMeV2T9)       * fullsolY.y[5,i]*fullsolY.y[7,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])

Be7rates[2]
Be7reverserates[5]

mainLi7d=np.array([Li7reverserates[6].eval(fullsolY.y[0,i]*TMeV2T9*1e9) * fullsolY.y[3,i]*fullsolY.y[9,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])
Alter_mainLi7d=np.array([AlterLi7d(fullsolY.y[0,i]*TMeV2T9)             * fullsolY.y[3,i]*fullsolY.y[9,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])

mainBe7=np.array([Be7rates[2].eval(fullsolY.y[0,i]*TMeV2T9*1e9) * fullsolY.y[6,i]*fullsolY.y[7,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])
Alter_mainBe7=np.array([AlterBe7(fullsolY.y[0,i]*TMeV2T9)       * fullsolY.y[6,i]*fullsolY.y[7,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])

mainBe7d=np.array([Be7reverserates[5].eval(fullsolY.y[0,i]*TMeV2T9*1e9) * fullsolY.y[2,i]*fullsolY.y[11,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])
Alter_mainBe7d=np.array([AlterBe7d(fullsolY.y[0,i]*TMeV2T9)             * fullsolY.y[2,i]*fullsolY.y[11,i]*rho_bY_cgs(fullsolY.y[:,i]) for i in range(len(fullsolY.t))])


plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,mainLi7,label='H3 + He4 ⟶ Li7')
plt.plot((fullsolY.t+t_cut)/timeunit,Alter_mainLi7,'--',label='Alter H3 + He4 ⟶ Li7')

plt.legend(loc='lower right')
plt.xscale('log')
plt.yscale('log')
#plt.ylim(-1e-19,1e-19)
#plt.xlim(5e4,7e4)

In [ ]:
plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,mainBe7d,label='Be7 + n ⟶ p + Li7')
plt.plot((fullsolY.t+t_cut)/timeunit,Alter_mainBe7d,'--',label='Alterbbn Be7 + n ⟶ p + Li7')

plt.legend()
plt.xscale('log')

In [ ]:

plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,(Alter_mainBe7-mainBe7)/mainBe7,label='He3 + He4 ⟶ Be7')
plt.plot((fullsolY.t+t_cut)/timeunit,(Alter_mainBe7d-mainBe7d)/mainBe7d,label='Be7 + n ⟶ p + Li7')

plt.axhline(0,color='k',linestyle=':',label='0')
plt.legend(loc='lower right')
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('(dY/dt - dY/dt_alterbbn)/dY/dt')
plt.ylim(-0.5,0.5)
#plt.yscale('log')
#plt.ylim(1e-90,4e-9)

In [ ]:
plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,(Alter_mainLi7-mainLi7)/mainLi7,label='H3 + He4 ⟶ Li7')
plt.plot((fullsolY.t+t_cut)/timeunit,(Alter_mainLi7d-mainLi7d)/mainLi7d,label='Be7 + n ⟶ p + Li7')

plt.axhline(0,color='k',linestyle=':',label='0')
plt.legend(loc='lower right')
plt.xscale('log')
plt.xlabel('time in seconds')
plt.ylabel('(dY/dt - dY/dt_alterbbn)/dY/dt')
plt.ylim(-0.5,0.5)
#plt.yscale('log')
#plt.ylim(1e-90,4e-9)

In [ ]:
plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,Alter_mainLi7/mainLi7,label='H3 + He4 ⟶ Li7')


plt.legend(loc='lower right')
plt.xscale('log')
#plt.yscale('log')
#plt.ylim(1e-90,4e-9)

In [ ]:
def n_p__d(T9):
    # n + p --> d
    rate = 0.0
    T913=T9**(1/3)
    T953=T9**(5/3)
    lnT9=np.log(T9)
    # an06n
    rate += np.exp(  12.3687 + -2.70618*T913
                  + 0.11718*T9 + -0.00312788*T953 + 0.469127*lnT9)
    # an06n
    rate += np.exp(  10.7548 + -2.30472*T913 + -0.887862*T9 + 0.137663*T953)
    # an06n
    rate += np.exp(  8.84688 + -0.0102082*T913  + -0.0893959*T9 + 0.00696704*T953 + 1.0*lnT9)

    return rate

def Alter_n_p__d(T9):
    if(T9<=1.5):
        res=44216.*(1+3.75191*T9+1.92934*T9*T9+0.746503*T9*T9*T9+0.0197023*pow(T9,4.)+3.00491e-6*pow(T9,5.))/(1+5.4678*T9+5.62395*T9*T9+0.489312*T9*T9*T9+0.00747806*pow(T9,4.))
    else:
        res=(1.-np.sqrt(T9)*0.8504+T9*0.4895-pow(T9,3./2.)*0.09623+T9*0.008471*T9-T9*2.8e-4*pow(T9,3./2.))*47420.
    return res


In [ ]:
plt.figure()
plt.plot((fullsolY.t+t_cut)/timeunit,[n_p__d(T*TMeV2T9) for T in fullsolY.y[0]],label='d + p ⟶ p + Li7')
plt.plot((fullsolY.t+t_cut)/timeunit,[Alter_n_p__d(T*TMeV2T9) for T in fullsolY.y[0]],'--',label='Alterbbn d + p ⟶ p + Li7')

plt.legend()
plt.xscale('log')

In [ ]:
n_p__d(1)

In [ ]:
fullsolY.y[0]